In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import os
import re
from argparse import Namespace
from pathlib import Path
from typing import Dict, Type, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import yaml
from torch.utils.data import DataLoader

from double_jig_gen.data import (
    ABCDataset,
    get_folkrnn_dataloaders,
    pad_batch,
    get_oneills_dataloaders
)
from double_jig_gen.tokenizers import Tokenizer
from double_jig_gen.models import SimpleRNN, Transformer
from double_jig_gen.utils import get_model_from_checkpoint

logging.basicConfig()
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel("DEBUG")

In [ ]:
def _get_most_recent_path(paths):
    """Returns the most recently created path from a list of paths.

    Args:
        paths: a list of paths to check.

    Returns:
        the most recently created path.
    """
    return max(paths, key=os.path.getctime)

In [ ]:
expt_ids = list(range(23, 31))
# expt_ids.remove(25)

In [ ]:
def get_args(expt_dirpath):    
    checkpoint_dirpath = Path(expt_dirpath, "checkpoints")
    ckpt_paths = [
        path for path in checkpoint_dirpath.iterdir() if str(path).endswith(".ckpt")
    ]
    latest_ckpt_path = _get_most_recent_path(ckpt_paths)

    experiment_args_path = Path(expt_dirpath, "experiment_args.yaml")
    # The yaml file has lowcase trainer in tag:
    # python/name:pytorch_lightning.trainer.trainer._gpus_arg_default
    # so loading fails with SafeLoader, have to use BaseLoader
    # args = pl.core.saving.load_hparams_from_yaml(str(experiment_args_path))
    with open(str(experiment_args_path), 'r') as fh:
        args_dict = yaml.load(fh, Loader=yaml.BaseLoader)
    args_dict['latest_checkpoint'] = latest_ckpt_path
    args_dict["checkpoint_epoch"] = int(
        ''.join(re.findall(r"\d+", str(latest_ckpt_path.name)))
    )
    args = Namespace()
    vars(args).update(args_dict)
    return args

In [ ]:
scratch_path = "/disk/scratch_fast"
expt_dirpath = Path(f"{scratch_path}/s0816700/logs/lightning_logs")
expt_dirs = [
    Path(expt_dirpath, f"version_{expt_id}") for expt_id in expt_ids
]
args = {
    str(dirpath): vars(get_args(dirpath)) for dirpath in expt_dirs
}

In [ ]:
expt_configs = pd.DataFrame.from_dict(args, orient='index')

In [ ]:
cols = [
    "dataset",
    "checkpoint_epoch",
    "early_stopping_patience",
    "val_prop", 
    "val_shuffle",
    "batch_size",
    "model_load_from_checkpoint",
    "latest_checkpoint",
    "seed",
    "model"
]

In [ ]:
expt_configs["model"] = expt_configs.latest_checkpoint.apply(
    lambda ckpt_path: SimpleRNN.load_from_checkpoint(checkpoint_path=str(ckpt_path))
)

In [ ]:
expt_configs[cols]

In [ ]:
trn, vld, tst = get_oneills_dataloaders(
    "/disk/scratch_fast/s0816700/data/oneills/oneills_reformat.abc",
    "/disk/scratch_fast/s0816700/data/folk-rnn/data_v3_vocabulary.txt",
    batch_size=16,
    num_workers=4,
    pin_memory=True,
)

In [ ]:
lightning_trainer = pl.Trainer(
    gpus='7,',
)

In [ ]:
model = SimpleRNN(
    rnn_type="LSTM",
    ntoken=106,
    ninp=256,
    nhid=512,
    nlayers=3,
    model_batch_size=16,
    dropout=.6,
    embedding_padding_idx=0,
)

In [ ]:
def get_avg_loss_per_token(model, dataloader, device):
    model = model.to(device)
#     print(dict(model.named_parameters()))
    model.eval()
    loss_total = 0
    with torch.no_grad():
        for padded_batch, seq_lens in dataloader:
            padded_batch = padded_batch.to(device)
            outputs = model(padded_batch, seq_lens)
            loss_total += model.loss(outputs, padded_batch).item()
    return loss_total / len(tst)

In [ ]:
get_avg_loss_per_token(model, tst, 'cuda')

In [ ]:
test_res = {}
for log_dir, log_data in expt_configs.iterrows():
    model = log_data.model
    test_res[log_dir] = get_avg_loss_per_token(model, tst, 'cuda')
    
#     model.to('cuda')
#     print({name: data.shape for name, data in model.named_parameters()})
    # WEIRD CUDA ERRORS...
#     test_res[ckpt_path] = lightning_trainer.test(
#         model,
#         test_dataloaders=tst,
#         ckpt_path=ckpt_path,
#     )
#     test_res[ckpt_path] = get_avg_loss_per_token(model, tst, device='cuda')

In [ ]:
expt_configs["test_loss"] = pd.Series(test_res)

In [ ]:
cols = [
    "test_loss",
    "checkpoint_epoch",
    "val_prop", 
    "val_shuffle",
    "batch_size",
    "model_load_from_checkpoint",
    "seed",
]
expt_configs[cols]